In [0]:
#libraries to be used
import numpy as np
import math
import copy
import heapq

In [0]:
class PriorityQueue:
    def __init__(self):
        self.elements = []
    
    def empty(self):
        return len(self.elements) == 0
    
    def put(self, priority, item):
        heapq.heappush(self.elements, (priority, item))
    
    def get(self):
        return heapq.heappop(self.elements)
    


In [0]:
#Grid class with all tiles positions and exact distance from
#the start (gVal) and estimate distance to the goal plus gVal (fVal)
class gridObj:
  def __init__(self,tiles,index,gVal,fVal,parent):
    self.tiles = tiles
    self.index = index
    self.gVal = gVal
    self.fVal = fVal
    self.parent = parent
  
  def __lt__(self,other):
    valueF = self.gVal + self.fVal
    valueO = other.gVal + other.fVal
    if(valueF>valueO):
      return 0
    else:
      return 1
  
  def setgVal(self,gVal):
    self.gVal = gVal
  
  def setParent(self,parent):
    self.parent = parent



In [0]:
#Creates a grid if it doesn't exist already and add it to the list of grids, else return the existing grid 
def createGrid(current,gVal,fVal,grids,parent):
  if len(grids)>0:
    for i in grids:
      if ((i.tiles==current).all()):
        return i
  tempGrid = gridObj(current, len(grids), gVal, fVal, parent)
  grids.append(tempGrid)
  return tempGrid



In [0]:
#This function takes the value of a grid position and 
# return an npArray with the proper position in the goal Grid 
def valToPos(val):
    pos = np.zeros((2))
    if (val==0):
      x = 2
      y = 2
    else:  
      if val <= 3 and val!=0 :
        x = 0
        y = val -1 
      elif val <= 6 :
        x = 1
        y = val -4
      elif val<=8:
        x = 2
        y = val - 7
      
    pos[0] = int (x)
    pos[1] = int (y)
    return pos

In [0]:
def returnEmpty(grid):
  empty = np.zeros((2))
  for i in range(0,3):
    for j in range(0,3):
      if (grid[i,j]==0):
        empty[0] = i
        empty[1] = j
        return empty #Esa linea equivale a un break
  return empty

In [0]:
#return a list of the neighbors for a grid, creates the necessary grids also
def neighbors (grid,gVal,grids,parent):
  empty = returnEmpty(grid) #Get the position of the empty spot 
  x = empty[0]
  y = empty[1]
  x = int(x)
  y = int(y)
  neighbors = []
  
  if (x+1<3):
    tempGrid = np.copy(grid) #copy the current grid
    tempPos = grid[x+1,y] #Save one of the positions to change
    tempGrid[x+1,y] = tempGrid[x,y] #change the value of the position saved before
    tempGrid[x,y] = tempPos #Change the value of the other position
    neighbors.append(createGrid(tempGrid, gVal+1, heurDist(tempGrid), grids, parent))
  if (x-1>=0):
    tempGrid = np.copy(grid) #copy the current grid
    tempPos = grid[x-1,y] #Save one of the positions to change
    tempGrid[x-1,y] = tempGrid[x,y] #change the value of the position saved before
    tempGrid[x,y] = tempPos #Change the value of the other position
    neighbors.append(createGrid(tempGrid,gVal+1,heurDist(tempGrid), grids, parent))
  if (y+1<3):
    tempGrid = np.copy(grid) #copy the current grid
    tempPos = grid[x,y+1] #Save one of the positions to change
    tempGrid[x,y+1] = tempGrid[x,y] #change the value of the position saved before
    tempGrid[x,y] = tempPos #Change the value of the other position
    neighbors.append(createGrid(tempGrid,gVal+1,heurDist(tempGrid), grids, parent))
  if (y-1>=0):
    tempGrid = np.copy(grid) #copy the current grid
    tempPos = grid[x,y-1] #Save one of the positions to change
    tempGrid[x,y-1] = tempGrid[x,y] #change the value of the position saved before
    tempGrid[x,y] = tempPos #Change the value of the other position
    neighbors.append(createGrid(tempGrid,gVal+1,heurDist(tempGrid), grids, parent))
  return neighbors


In [0]:
#iterates through the grid and accumulate the distance between
#every tile in the current grid to the correct tile position in the final grid
def heurDist(grid):
  dist = 0
  if(grid[0,0]!=1):
    dist = 5
  val = 1
  for i in range(0,3):
    for j in range(0,3):
      pos = valToPos(grid[i,j])
      dist = dist + (abs(pos[0]-i) + abs(pos[1]-j))
      val = val + 1
  #print(dist)
  dist = dist + linearConflict(grid) - 1
  if(dist<0):
    dist = 0
  return dist

In [0]:
def linearConflict(grid):
  acum = 0
  for i in grid:
    if(level(i[0])==level(i[1]) and i[1]<i[0]):
      acum = acum + 1
      if(i[1]==0):
        acum = acum - 1
    if(level(i[0])==level(i[2]) and i[2]<i[0]):
      acum = acum + 1  
      if(i[2]==0):
        acum = acum - 1
    if(level(i[1])==level(i[2]) and i[2]<i[1]):
      acum = acum + 1
      if(i[2]==0):
        acum = acum - 1
  return acum

     




In [0]:
def level(val):
  if val == 0 or val > 6:
    return 2
  elif val>=1 and val <=3:
    return 0
  else:
    return 1

In [0]:
#function to find an npArray element on a list 
def findQueu(element,listF):
  tempList = copy.deepcopy(listF)
  while not tempList.empty():
    obj = tempList.get()
    obj = obj[1]
    if ((element.tiles==obj.tiles).all()):
      return True
  return False

In [0]:
def findList(tiles,listC):
  for element in listC:
    if ((element==tiles).all()):
      return True
  return False

In [0]:
def delList(tiles,listC):
  index = 0
  for element in listC:
    if ((element==tiles).all()):
      return index
  index = index + 1  
  return False

In [0]:
def a_star_search(start, goal, grids):
  frontier = PriorityQueue() #PriorityQueue with the unexplored grids
  close = []
  priority = start.gVal + start.fVal
  frontier.put(priority, start) #First value of the frontier is the first grid
  cameFrom = [] #The index of a position in cameFrom indicates the referenced grid, the value in that index position is the index to the grid which current came from 
  currentCost = [] #Index in currentCost refers to a grid Index, the value in that index of the list is the distance from the starter grid.
  cameFrom.insert(start.index,start.tiles)
  currentCost.insert(start.index,0)
  steps = 1
  pace = 100
  while not frontier.empty() :
    current = frontier.get() #get the grid with the lowest fVal 
    current = current[1]
    close.append(current.tiles)
    priority = current.gVal + current.fVal
    # frontier.put(priority, current)
    if (current.tiles==goal).all() :
      return current
    #create the list of neighbors
    neighborList = neighbors(current.tiles, current.gVal, grids,current)
    #iterate through neighbors to find the best path
    for neighbor in neighborList:
      #If it is already in the closed set, skip it
      if(not findList(neighbor.tiles,close)):
        #if neighbor on frontier
        if(findQueu(neighbor,frontier)):
          
          tempgVal = current.gVal + 1
          if(tempgVal<neighbor.gVal):
            neighbor.setgVal(tempgVal)
            neighbor.setParent(current)
        else:
          priority = neighbor.gVal + neighbor.fVal
          frontier.put(priority, neighbor)
          
    steps = steps + 1
  print('Sin solucion')  
  return []
        
      
    
    

In [0]:
def path(finalGrid):
  pathFinal = []
  if(finalGrid != []):
    if(finalGrid != None):
      pathFinal.append(finalGrid)
      if(finalGrid.parent != None):
        finalGrid = finalGrid.parent 
  return pathFinal

In [176]:
global index 
index = 0
grids = []
grid =np.array([[3, 7, 0],[2, 1, 4],[8, 5, 6],])
goal =np.array([[1, 2, 3],[4, 5, 6],[7, 8, 0],])
createGrid(grid, 0, valF, grids, None)
final = a_star_search(grids[0], goal,grids)
path = []
path.append(final.tiles)
while(final.parent != None):
  path.append(final.parent.tiles)
  final = final.parent
path.reverse()
print('numero de movimientos fue: ' + str(len(path) - 1))
print('')
print('camino')
print('')
for i in path:
  print(i)
  print('')

numero de movimientos fue: 23

camino

[[3 7 0]
 [2 1 4]
 [8 5 6]]

[[3 0 7]
 [2 1 4]
 [8 5 6]]

[[0 3 7]
 [2 1 4]
 [8 5 6]]

[[2 3 7]
 [0 1 4]
 [8 5 6]]

[[2 3 7]
 [1 0 4]
 [8 5 6]]

[[2 3 7]
 [1 4 0]
 [8 5 6]]

[[2 3 0]
 [1 4 7]
 [8 5 6]]

[[2 0 3]
 [1 4 7]
 [8 5 6]]

[[0 2 3]
 [1 4 7]
 [8 5 6]]

[[1 2 3]
 [0 4 7]
 [8 5 6]]

[[1 2 3]
 [4 0 7]
 [8 5 6]]

[[1 2 3]
 [4 5 7]
 [8 0 6]]

[[1 2 3]
 [4 5 7]
 [0 8 6]]

[[1 2 3]
 [0 5 7]
 [4 8 6]]

[[1 2 3]
 [5 0 7]
 [4 8 6]]

[[1 2 3]
 [5 7 0]
 [4 8 6]]

[[1 2 3]
 [5 7 6]
 [4 8 0]]

[[1 2 3]
 [5 7 6]
 [4 0 8]]

[[1 2 3]
 [5 0 6]
 [4 7 8]]

[[1 2 3]
 [0 5 6]
 [4 7 8]]

[[1 2 3]
 [4 5 6]
 [0 7 8]]

[[1 2 3]
 [4 5 6]
 [7 0 8]]

[[1 2 3]
 [4 5 6]
 [7 8 0]]

